# PIPELINES FORMATION FOR CLEANING , PREPROCESSING AND TRANSFORMATION

In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


# 3.1 Data Cleaning & Standardisation

In [9]:
import pandas as pd
import numpy as np

# Load datasets
bio_df = pd.read_csv(r"C:\Users\Akash\Desktop\api_data_aadhar_biometric_0_500000.csv")
demo_df = pd.read_csv(r"C:\Users\Akash\Desktop\api_data_aadhar_demographic_0_500000.csv")
enroll_df = pd.read_csv(r"C:\Users\Akash\Desktop\api_data_aadhar_enrolment_0_500000.csv")

# Standardisation mapping for legacy state / city names
state_mapping = {
    'Orissa': 'Odisha',
    'Pondicherry': 'Puducherry',
    'Gurgaon': 'Gurugram',
    'Bangalore': 'Bengaluru'
}

# Apply cleaning and standardisation to all datasets
dfs = [bio_df, demo_df, enroll_df]

for df in dfs:
    # Clean column names
    df.columns = df.columns.str.strip().str.lower()

    # Standardise state names
    if 'state' in df.columns:
        df['state'] = df['state'].replace(state_mapping)

    # Parse dates explicitly (DD-MM-YYYY) to avoid ambiguity
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(
            df['date'],
            format='%d-%m-%Y',
            errors='coerce'
        )

    # Drop records without pincodes (critical for geo-spatial analysis)
    if 'pincode' in df.columns:
        df.dropna(subset=['pincode'], inplace=True)


# 3.1.5 Dataset Integration (merged_df)

In [14]:
# Aggregate biometric data
bio_agg = bio_df.groupby(
    ['state', 'district', 'pincode'],
    as_index=False
).agg(
    bio_updates=('date', 'count')
)

# Aggregate demographic data
demo_agg = demo_df.groupby(
    ['state', 'district', 'pincode'],
    as_index=False
).agg(
    demo_updates=('date', 'count')
)

# Aggregate enrolment data
enroll_agg = enroll_df.groupby(
    ['state', 'district', 'pincode'],
    as_index=False
).agg(
    enrol_updates=('date', 'count')
)


In [15]:
merged_df = bio_agg.merge(
    demo_agg,
    on=['state', 'district', 'pincode'],
    how='outer'
).merge(
    enroll_agg,
    on=['state', 'district', 'pincode'],
    how='outer'
)


In [17]:
merged_df.fillna(0, inplace=True)

merged_df['total_updates'] = (
    merged_df['bio_updates']
    + merged_df['demo_updates']
    + merged_df['enrol_updates']
)


In [18]:
print(merged_df.shape)
print(merged_df.head())


(30301, 7)
                       state  district  pincode  bio_updates  demo_updates  \
0  Andaman & Nicobar Islands  Andamans   744101         20.0          16.0   
1  Andaman & Nicobar Islands  Andamans   744103         17.0          12.0   
2  Andaman & Nicobar Islands  Andamans   744105         16.0          16.0   
3  Andaman & Nicobar Islands  Andamans   744106         14.0          14.0   
4  Andaman & Nicobar Islands  Andamans   744107          9.0           9.0   

   enrol_updates  total_updates  
0            8.0           44.0  
1            9.0           38.0  
2            7.0           39.0  
3            4.0           32.0  
4            5.0           23.0  


# 3.2 Outlier detection and geospatial clustering (using pincode)

In [21]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

# 3.2.1 Outlier Detection (Low & High Activity Zones)

In [22]:
# Calculate IQR for total_updates
Q1 = merged_df['total_updates'].quantile(0.25)
Q3 = merged_df['total_updates'].quantile(0.75)
IQR = Q3 - Q1

# Define thresholds
low_threshold = Q1 - 1.5 * IQR
high_threshold = Q3 + 1.5 * IQR

# Identify outliers
low_activity_outliers = merged_df[
    merged_df['total_updates'] < low_threshold
]

high_activity_outliers = merged_df[
    merged_df['total_updates'] > high_threshold
]

print("Low-Activity Outlier Pincodes:", low_activity_outliers.shape[0])
print("High-Activity Outlier Pincodes:", high_activity_outliers.shape[0])


Low-Activity Outlier Pincodes: 0
High-Activity Outlier Pincodes: 8


# Prepare data for clustering (PINCODE level)

In [24]:
features = merged_df[['total_updates']].values

# Scale features

In [25]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# K-Means Clustering

In [26]:
# 4 clusters: Very Low, Low, Medium, High activity
kmeans = KMeans(n_clusters=4, random_state=42)
merged_df['service_cluster'] = kmeans.fit_predict(scaled_features)

# Identify Service-Starved Zones

In [27]:
# Compute mean activity per cluster
cluster_means = merged_df.groupby('service_cluster')['total_updates'].mean()

# Cluster with minimum activity
low_service_cluster = cluster_means.idxmin()

# High priority pincodes
high_priority_pincodes = merged_df.loc[
    merged_df['service_cluster'] == low_service_cluster,
    'pincode'
].unique()

print("\nHigh Priority (Service-Starved) Pincodes:")
print(high_priority_pincodes)



High Priority (Service-Starved) Pincodes:
[744112 744206 744211 ... 721135 700149 743513]


# Summary Table (REPORT-READY)

In [28]:
cluster_summary = merged_df.groupby('service_cluster').agg(
    pincodes=('pincode', 'nunique'),
    avg_updates=('total_updates', 'mean'),
    min_updates=('total_updates', 'min'),
    max_updates=('total_updates', 'max')
)

print("\nCluster Summary:")
print(cluster_summary)



Cluster Summary:
                 pincodes  avg_updates  min_updates  max_updates
service_cluster                                                 
0                    5482    37.679164         23.0         48.0
1                    8221    78.010357         69.0        156.0
2                    5169     7.442419          1.0         22.0
3                    8262    59.036859         49.0         68.0
